# Setup

Run this cell once at the start

In [ ]:
# redcap_api.ipynb, a Python script for getting data from ORCID and loading it into a RedCap form.
version = '0.1'
created = '2022-02-25'

# (c) 2022 Vanderbilt University. This program is released under a GNU General Public License v3.0 http://www.gnu.org/licenses/gpl-3.0
# Author: Steve Baskauf

# -------------
# Configuration
# -------------

# Module imports

import requests
import json
import hashlib
from pathlib import Path

# Global variables
endpoint_url = 'https://redcap.vanderbilt.edu/api/'

# ---------------------

# Function definitions

# value of directory should be either 'home' or 'working'
def load_credential(filename, directory):
    """Retrieve the raw token text from a plain text file."""
    cred = ''
    # to change the script to look for the credential in the working directory, change the value of home to empty string
    if directory == 'home':
        home = str(Path.home()) #gets path to home directory; supposed to work for Win and Mac
        credential_path = home + '/' + filename
    else:
        directory = 'working'
        credential_path = filename
    try:
        with open(credential_path, 'rt', encoding='utf-8') as file_object:
            cred = file_object.read()
    except:
        print(filename + ' file not found - is it in your ' + directory + ' directory?')
        exit()
    return(cred)

def retrieve_api_data(token, fields):
    """Sends GET request to API and convert result to data structure."""
    data = {
        'token': token,
        'format': 'json',
        'returnFormat': 'json'
    }
    data.update(fields)
    response = requests.post(endpoint_url,data=data)
    status = response.status_code
    print('status:', status)
    if status == 200:
        try:
            data = response.json()
            return status, data
        except:
            print(response.text)
            return 0, {}
    else:
        print(response.text)
        return status, {}

# ---------------------

# Load token from file

token = load_credential('redcap_test_token.txt', 'home')


# Set up specific data fields

The following cells can be run individually to do different kinds of API interactions.
See https://redcap.vanderbilt.edu/api/help/ for more info. Clicking on the `API Playground` link will let you interactively build the request JSON for different languages.

In [ ]:
# Retrieve some records
data_fields = {
    'content': 'record',
    'action': 'export',
    'type': 'flat',
    'csvDelimiter': '',
    'records[0]': '1',
    'fields[0]': 'record_id',
    'fields[1]': 'orcid',
    'fields[2]': 'name',
    'fields[3]': 'doi',
    'fields[4]': 'citation',
    'fields[5]': 'type',
    'fields[6]': 'form_1_complete',
    'forms[0]': 'form_1',   
    'rawOrLabel': 'raw',
    'rawOrLabelHeaders': 'raw',
    'exportCheckboxLabel': 'false',
    'exportSurveyFields': 'false',
    'exportDataAccessGroups': 'false'
}


In [ ]:
# Successful test of writing
# You can send multiple records at once, but the 
# hashlib.sha1().hexdigest()[:16]
# seems to result in duplicates if you do it for two records. Not sure why...

fields =  [
    {
    'record_id': hashlib.sha1().hexdigest()[:16],
    "orcid": "1234-2222-7777-2678",
    "name": "Jing Ping",
    "doi": "10.1234/j.mol.stuff.123",
    "citation": "J. Zhang. 2021. My better article. J.Whatever 2:12-13",
    "type": " Journal article",
    "form_1_complete": "2"  
},
{
    'record_id': '1000',
    "orcid": "1234-2222-333-2678",
    "name": "Xiang Zeng",
    "doi": "10.1234/j.mol.stuff.365",
    "citation": "J. Zhang. 2021. My best article. J.Whatever 2:12-13",
    "type": " Journal article",
    "form_1_complete": "2"  
}
]

data_fields = {
    'content': 'record',
    'type': 'flat',
    'data': json.dumps(fields)
}


In [ ]:
# Get project information
data_fields = {'content': 'project'}


In [ ]:
# Get field names from forms
data_fields = {'content': 'exportFieldNames'}


# Main script

Run after defining `data_fields` in one of the preceeding cells

In [ ]:
status, data = retrieve_api_data(token, data_fields)
if status == 200:
    print(json.dumps(data, indent = 2))
else:
    print('no data retrieved')
